In [1]:
import pandas as pd
import statsmodels.api as sm

In [4]:
data = pd.read_parquet('../../data/curated/clean_stats_13-22.parquet')
data.set_index('player_id', inplace=True)

In [5]:
features = []
with open('final_features.txt', 'r') as f:
    for line in f.readlines():
        features.append(line.strip())

target = 'brownlow_votes'

In [6]:
X_train, y_train = data.query('season < 2022')[features], data.query('season < 2022')[target]
X_test, y_test = data.query('season == 2022')[features], data.query('season == 2022')[target]

In [7]:
model = sm.MNLogit(y_train, sm.add_constant(X_train)).fit(disp=0, maxiter=10_000)

In [8]:
predictions = model.predict(sm.add_constant(X_test))

In [20]:
df_2022 = data.query('season == 2022')[['match_id', 'match_round', 'player_first_name', 'player_last_name', 'player_team']]

In [23]:
out_df = pd.concat([df_2022, predictions], axis=1)

In [24]:
out_df

,match_id,match_round,player_first_name,player_last_name,player_team,0,1,2,3
player_id,,,,,,,,,
11904,16117,1,Tom,Liberatore,Western Bulldogs,0.998700,0.000973,0.000299,0.000028
11945,16117,1,Steven,May,Melbourne,0.995085,0.003862,0.000924,0.000128
11972,16117,1,Max,Gawn,Melbourne,0.940318,0.031202,0.023883,0.004596
12015,16117,1,Tom,McDonald,Melbourne,0.998167,0.001388,0.000399,0.000046
12034,16117,1,Adam,Tomlinson,Melbourne,0.999429,0.000466,0.000096,0.000008
...,...,...,...,...,...,...,...,...,...
12863,16346,23,Logan,McDonald,Sydney,0.996589,0.002546,0.000776,0.000090
12864,16346,23,Errol,Gulden,Sydney,0.982434,0.011877,0.004978,0.000710
12945,16346,23,Mitch,Owens,St Kilda,0.997681,0.001821,0.000448,0.000050


In [ ]:
out_df.to_parquet('../../data/curated/2022_vote_probs.parquet')